## Function Calling / Tool Calling

In [1]:
from openai import AzureOpenAI

client = AzureOpenAI(api_version="2024-12-01-preview")

In [ ]:
response = client.chat.completions.create(model='telcogpt',
                                          messages=[{'role':'user',
                                                    'content':'Explain Quantum Computing in 2 lines'}],
                                        temperature=0.2)

print(response.to_json(indent=2))


In [9]:

import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


In [28]:
import wikipedia
def get_wkipedia_summary(query:str)->str:
    response = wikipedia.summary(query)
    return response



In [30]:
get_wkipedia_summary("capital city of Sikkim")

"Gangtok (Nepali: gāntok, pronounced [gantok], Sikkimese: [ɡaŋt̪ʰòk]) is the capital and the most populous city of the Indian state of Sikkim. The seat of eponymous district, Gangtok is in the eastern Himalayan range, at an elevation of 1,650 m (5,410 ft). The city's population of 100,000 consists of the three Sikkimese ethnicities: the Bhutias, Lepchas, Nepalis as well as plainsmen from other states of India. Within the higher peaks of the Himalayas and with a year-round mild temperate climate, Gangtok is at the centre of Sikkim's tourism industry.\nGangtok rose to prominence as a popular Buddhist pilgrimage site after the construction of the Enchey Monastery in 1840. In 1894, the ruling Sikkimese Chogyal, Thutob Namgyal, transferred the capital to Gangtok. In the early 20th century, Gangtok became a major stopover on the trade route between Lhasa in Tibet and cities such as Kolkata (then Calcutta) in British India. After India won its independence from the British Empire in 1947, Sik

In [10]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'clear sky',
 'temperature': 303.26,
 'unit': 'kelvin'}

In [31]:
# name, description, parameters (also desc for parameters)

tools = [{"type":"function",
          "function":{
              "name":"get_current_weather_info",
              "description":"this function is used to get the current weather information for a given city",
              "parameters":{
                  "type":"object",
                  "properties":{"city":{"type":"string","description":"name of location/city, e.g. mumbai, new york"}}

              },
              "required":['city'],
          }},
          {"type":"function",
          "function":{
              "name":"get_wikipedia_summary",
              "description":"this function is used to get information about locaiton, places, people, history, from wkipedia, given the topic",
              "parameters":{
                  "type":"object",
                  "properties":{"query":{"type":"string","description":"topic to search on wikipedia e.g. capital of India"}}

              },
              "required":['query'],
          }},
          
          ]

In [24]:
def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful asistant"},
                {"role":"user","content":prompt}]
    first_response = client.chat.completions.create(model='telcogpt',
                                                    messages=messages,
                                                    temperature=0.1,tools=tools,
                                                    tool_choice='auto')
    
    if first_response.choices[0].message.tool_calls:
        print(first_response.choices[0].message.tool_calls)
        tool_calls = first_response.choices[0].message.tool_calls
        tool_map = {"get_current_weather_info":get_current_weather}
        messages.append(first_response.choices[0].message)
        for tool in tool_calls:
            tool_name = tool.function.name # fetching the function name (metadata reference)
            tool_args = json.loads(tool.function.arguments) # fetching funciton arguments
            function_to_ex = tool_map[tool_name] # getting the tool name
            tool_output = function_to_ex(**tool_args) # executing the function
            messages.append({"tool_call_id":tool.id,"role":"tool","name":tool_name,
                             "content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model="telcogpt",messages=messages,
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [25]:
resp = generate_response("what is the current weather in mumbai")
print(resp)

[ChatCompletionMessageToolCall(id='call_Xnnb605u70kq2nPwMbAxCLpz', function=Function(arguments='{"city":"mumbai"}', name='get_current_weather_info'), type='function')]
The current weather in Mumbai is clear sky with a temperature of approximately 303.26 K (about 30.1 °C).


In [26]:
resp = generate_response("what is quantum computing")
print(resp)

Quantum computing is a type of computation that takes advantage of the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key concepts:

1. **Quantum Bits (Qubits)**: Unlike classical bits, which can be either 0 or 1, qubits can exist in a state of superposition, meaning they can be both 0 and 1 simultaneously. This allows quantum computers to process a vast amount of information at once.

2. **Superposition**: This principle allows qubits to be in multiple states at the same time. When a qubit is in superposition, it can perform multiple calculations simultaneously, which can lead to faster problem-solving for certain tasks.

3. **Entanglement**: Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This property can be used to link qubits in complex ways, enabling more powerful computations.

4. **Quantum Gates**: Quantum comp

In [27]:
resp = generate_response("what is the current weather in mumbai and London")
print(resp)

[ChatCompletionMessageToolCall(id='call_HByYo7sNzGOehdO1hq3Z8Bea', function=Function(arguments='{"city": "mumbai"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_yH1YpsDkIO1Ue3eo3ecNb7nc', function=Function(arguments='{"city": "London"}', name='get_current_weather_info'), type='function')]
The current weather is as follows:

- **Mumbai**: Clear sky with a temperature of approximately 303.26 K (about 30.1 °C).
- **London**: Few clouds with a temperature of approximately 285.04 K (about 12.9 °C).


In [44]:
import time

**Consider deloying another model like (gpt-40, gpt-4.1-mini) with token per minute rate via portal for this use case to avoid rate limit error.**

In [54]:
def generate_response2(prompt):

    messages = [{"role":"system","content":"you are a helpful asistant"},
                {"role":"user","content":prompt}]
    tool_map = {"get_current_weather_info":get_current_weather,
                "get_wikipedia_summary":get_wkipedia_summary}
    while True:
        first_response = client.chat.completions.create(model='telcogpt2',
                                                        messages=messages,
                                                        temperature=0.1,tools=tools,
                                                        tool_choice='auto')
        
        if first_response.choices[0].message.tool_calls:
            print(first_response.choices[0].message.tool_calls)
            tool_calls = first_response.choices[0].message.tool_calls
            
            messages.append(first_response.choices[0].message)
            for tool in tool_calls:
                tool_name = tool.function.name # fetching the function name (metadata reference)
                tool_args = json.loads(tool.function.arguments) # fetching funciton arguments
                function_to_ex = tool_map[tool_name] # getting the tool name
                tool_output = function_to_ex(**tool_args) # executing the function
                messages.append({"tool_call_id":tool.id,"role":"tool","name":tool_name,
                                "content":json.dumps(tool_output)})
            print("waiting for 30 seconds to Avoid rate limit error ",end="")
            for i in range(30):
                print(".",end="")
                time.sleep(1)
        else:
            break
    return first_response.choices[0].message.content

In [55]:
resp = generate_response2("what is the current weather in mumbai and London")
print(resp)

[ChatCompletionMessageToolCall(id='call_9RGgBDsieuhnolN92REpqjbK', function=Function(arguments='{"city": "mumbai"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_5jAYhKPyGojT444oE1peTWLB', function=Function(arguments='{"city": "london"}', name='get_current_weather_info'), type='function')]
waiting for 30 seconds to Avoid rate limit error ..............................The current weather in Mumbai is clear sky with a temperature of approximately 303.34 K. In London, the weather is few clouds with a temperature of approximately 285.76 K. If you want, I can also convert these temperatures to Celsius or Fahrenheit for you.


In [56]:
resp = generate_response2("what is the current weather in mumbai and mumbai is located in which state of India.")
print(resp)

[ChatCompletionMessageToolCall(id='call_AusGgYwMXN5IXDjulPNHEtbr', function=Function(arguments='{"city": "mumbai"}', name='get_current_weather_info'), type='function'), ChatCompletionMessageToolCall(id='call_hqX3RXgvh768BAvmjOlT2VqA', function=Function(arguments='{"query": "Mumbai"}', name='get_wikipedia_summary'), type='function')]
waiting for 30 seconds to Avoid rate limit error ..............................The current weather in Mumbai is clear sky with a temperature of approximately 303.34 Kelvin.

Mumbai is located in the state of Maharashtra, India. It is the capital city of Maharashtra and is known as the financial, commercial, and entertainment capital of South Asia.


In [57]:
resp = generate_response2("what is the current weather in capital city of Tripura.")
print(resp)

[ChatCompletionMessageToolCall(id='call_aQ1PRjon3eon3xvPBirOvGDO', function=Function(arguments='{"query":"capital of Tripura"}', name='get_wikipedia_summary'), type='function')]
waiting for 30 seconds to Avoid rate limit error ..............................[ChatCompletionMessageToolCall(id='call_FlxudIYkoHLsMvX9lmqvcGK5', function=Function(arguments='{"city":"Agartala"}', name='get_current_weather_info'), type='function')]
waiting for 30 seconds to Avoid rate limit error ..............................The current weather in Agartala, the capital city of Tripura, is scattered clouds with a temperature of approximately 33.5°C (306.63 K).
